# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
df = pd.read_csv('output_data/cities.csv')
df

,Unnamed: 0,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,namatanai,-3.6667,152.4333,299.05,81,95,0.62,PG,1619982983
1,1,mahebourg,-20.4081,57.7000,297.15,94,20,3.09,MU,1619982981
2,2,winnemucca,40.9730,-117.7357,287.15,44,90,5.14,US,1619982848
3,3,mar del plata,-38.0023,-57.5575,294.26,68,0,4.63,AR,1619982340
4,4,butaritari,3.0707,172.7902,301.60,74,100,6.10,KI,1619982984
5,5,cape town,-33.9258,18.4232,292.59,55,0,5.14,ZA,1619982911
6,6,chippewa falls,44.9369,-91.3929,294.15,43,75,2.57,US,1619982985
7,7,busselton,-33.6500,115.3333,286.48,93,58,6.88,AU,1619982985
8,8,olafsvik,64.8945,-23.7142,276.84,55,0,5.07,IS,1619982986
9,9,saskylakh,71.9167,114.0833,270.53,87,100,3.23,RU,1619982986


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key=g_key)
locations = df[['Lat', 'Long']].astype(float)
weight = df['Humidity'].astype(float)

In [20]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight,
                                dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
locations_narrow = df[(df['Max Temp']<299.817) & (df['Max Temp']>294.261) & (df['Wind Speed']<10) & (df['Cloudiness']<80)]
locations_narrow[['Lat', 'Long']]

,Lat,Long
1,-20.4081,57.7000
14,-23.1203,-134.9692
17,-14.2500,129.5500


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
locations_narrow['Hotel Name']=''

target_coordinates = locations_narrow[['Lat', 'Long']]
target_radius = 5000
target_type = 'hotel'

params = {
    'radius': target_radius,
    'types' : target_type,
    'key' : g_key,
    'keyword' :'hotel'
}


for index, row in target_coordinates.iterrows():
    lat = row['Lat']
    lon = row['Long']
    params['location'] = f"{lat},{lon}"
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    name_address = requests.get(base_url, params=params).json()
    print(name_address)
    
    try:
        locations_narrow.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print('Problem')
    

/Users/robthompson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.4407751, 'lng': 57.6714245}, 'viewport': {'northeast': {'lat': -20.43953072010728, 'lng': 57.67302222989272}, 'southwest': {'lat': -20.44223037989272, 'lng': 57.67032257010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Holiday Inn Mauritius Mon Tresor', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1836, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108550589248956408545">Krzysztof Wojciech Cembrzyński</a>'], 'photo_reference': 'ATtYBwIL2Hh3rpO-5T3X_66VC91n_x9qckw93dB4mq1IM9HuQrtKP0fK8ViPkPdFUjW_BPAVkc1pxzHu_-WDzxZzpNv9RTgtcLEppkflFE5D3VfZKOhf_tHBV2lgOz0EU8etn2u20hoKq3Hg42OWwgsrv2KX3uabetmAneg5hA2k14GYfcMG', 'width': 3264}], 'place_id': 'ChIJ89RtIu-KfCERwReKV5klpog', 'plus_code': {'compound_code': 'HM5C+MH Plaine Magnien, Mauritius', 'global_code': '5HFVHM5C+MH'}, 'rating': 4.1, 'referen

/Users/robthompson/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Problem
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -14.2376091, 'lng': 129.5192576}, 'viewport': {'northeast': {'lat': -14.23619557010728, 'lng': 129.5203622298928}, 'southwest': {'lat': -14.23889522989273, 'lng': 129.5176625701073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'TDC Village (Hundred Man Camp)', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106237541974471397837">Nicholas Cocks</a>'], 'photo_reference': 'ATtYBwKHcmTn4kHp5oYOsKyGxFD0RBJgxjk2ToRlDb5sMvXcYzn5lJt2RHYujzIDg8sesKfos2ofdlMonqcSVo1MblyPVzMwLDln7NALaZVNOVdMFY23rEdZVFbQGrP5RWWe2cLBp4fzVH-u-64NmvM62J4w129LiNcLdj2kOoprFSvP0i89', 'width': 4000}], 'place_id': 'ChIJ5xCIME9wliwRFR-WyMBAa8c', 'plus_code': {'compound_code': 'QG69+XP Wadeye, Northern Territ

In [23]:
print(locations_narrow)
hotel_df = locations_narrow[['Lat', 'Long', 'Hotel Name', 'City', 'Country']]
print(hotel_df)

    Unnamed: 0        City      Lat      Long  Max Temp  Humidity  Cloudiness  \
1            1   mahebourg -20.4081   57.7000    297.15        94          20   
14          14     rikitea -23.1203 -134.9692    298.44        76          63   
17          17  port keats -14.2500  129.5500    296.21        50           1   

    Wind Speed Country        Date                        Hotel Name  
1         3.09      MU  1619982981  Holiday Inn Mauritius Mon Tresor  
14        7.41      PF  1619982553                                    
17        3.47      AU  1619982989    TDC Village (Hundred Man Camp)  
        Lat      Long                        Hotel Name        City Country
1  -20.4081   57.7000  Holiday Inn Mauritius Mon Tresor   mahebourg      MU
14 -23.1203 -134.9692                                       rikitea      PF
17 -14.2500  129.5500    TDC Village (Hundred Man Camp)  port keats      AU


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]
print(locations)

        Lat      Long
1  -20.4081   57.7000
14 -23.1203 -134.9692
17 -14.2500  129.5500


In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))